In [1]:
import os
import subprocess
import datetime as dt
import itertools
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from MHW_daily_calcs import *

In [2]:
climyrs=[1993,2023]
smoothmethod='tri'
windowhalfwid=10

In [ ]:
fclim=xr.open_dataset(fnameOISSTDailyClim(climyrs[0],climyrs[-1]),decode_times=False)
climS=da.empty_like(fclim.variables['sst'].values)

In [ ]:
fclim['sst']

In [ ]:
smoothClim=trismooth(np.arange(0,365),fclim['sst'].values,L=windowhalfwid,periodic=True)

In [ ]:
np.shape(smoothClim)

In [ ]:
plt.plot(fclim['sst'][:,140,180],'r-')
plt.plot(smoothClim[:,140,180],'k-')

In [ ]:
fout=fnameOISSTDailyClimSmooth(climyrs[0],climyrs[-1],smoothmethod,windowhalfwid)
ds=xr.Dataset(data_vars={'sst':(['yearday','lat','lon'],smoothClim)},
                             coords={'yearday':np.arange(1,366),
                                     'lat':fclim.lat,
                                     'lon':fclim.lon})
ds.to_netcdf(fout,mode='w')

In [ ]:
fout


In [3]:
smoothedClim=True
if smoothedClim:
    climpath=fnameOISSTDailyClimSmooth(climyrs[0],climyrs[-1],smoothmethod,windowhalfwid)
else:
    climpath=climpath=fnameOISSTDailyClim(climyrs[0],climyrs[-1])
fclim=xr.open_dataset(climpath)

In [ ]:
for yrlims in ylimlistobs:
    print(yrlims)
    ifile=fnameOISSTDailyGrid2(yrlims)
    fsst=xr.open_dataset(ifile,decode_times=False,chunks={'time':365,})
    tdt=[dt.datetime(1978,1,1)+dt.timedelta(days=float(ii)) for ii in fsst.time.values]
    yd=[(el-dt.datetime(el.year-1,12,31)).days for el in tdt]
    # Loop over time
    sst_an=np.empty(np.shape(fsst.sst.values))
    for ind, iyd in enumerate(yd):
        if ind%100==0: print(ind)
        iyd = min(iyd,365)
        sst_an[ind,...] = fsst.sst.values[ind,...] - fclim.sst.values[iyd-1,...]
    for jj in range(0,180,60):
        fout=fnameOISSTAnom(yrlims, jj, smoothClim=True, meth=smoothmethod, win=windowhalfwid)
        dsout=xr.Dataset(data_vars={'sst_an':(['time','lat','lon'],sst_an[:,jj:jj+60,:])},
                         coords={'time':fsst.time,
                                 'lat':fsst.lat.isel(lat=slice(jj,jj+60)),
                                 'lon':fsst.lon})
        dsout.to_netcdf(fout,mode='w')
        print(fout)

[1991, 2000]
0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
2010
2040
2070
2100
2130
2160
2190
2220
2250
2280
2310
2340
2370
2400
2430
2460
2490
2520
2550
2580
2610
2640


In [ ]:
def _detrend(data):
    # remove trend along first dimension assuming evenly spaced data along that dimension
    # reshape to 2-d array to get fit, then transform fit back to original shape
    # return data with trend subtracted
    data=np.asarray(data)
    dshape = data.shape
    N=dshape[0]
    X=np.concatenate([np.ones((N,1)), np.expand_dims(np.arange(0,N),1)],1)
    M=np.prod(dshape,axis=0) // N # // is floor division
    newdata = np.reshape(data,(N, M)) 
    newdata = newdata.copy() # make sure a copy has been created
    # check there aren't extraneous nan's (besides fully masked land cells)
    if set(np.unique(np.sum(np.sum(np.isnan(fin.sst_an.data),axis=1),axis=0)))==set([0,np.prod(fin.sst_an.data.shape[:2])]):
        b, res, p, svs = np.linalg.lstsq(X,newdata,rcond=None)
    else: # extra nan's present; trigger annoying slow loop
        print('entering NaN loop')
        b=-9*np.ones((2,M))
        for ix in range(0,M):
            yy=newdata[:,ix]
            ind=~np.isnan(yy)
            xx=X[ind,:]
            yy=yy[ind].reshape((np.sum(ind),1))
            bb, res, p, svs = np.linalg.lstsq(xx,yy,rcond=None)
            b[:,ix]=bb[:,0]
        assert np.sum(b==-9)==0
    bshp = tuple([2]+list(dshape)[1:])
    b=np.reshape(b,bshp)
    trnd=np.arange(0,N).reshape((N,)+tuple(np.ones(len(dshape)-1,dtype=int))) * b[1,...].reshape((1,)+np.shape(b[1,...]))+\
                b[0,...].reshape((1,)+np.shape(b[0,:,:]))
    return data-trnd

In [ ]:
for jj in range(0,180,60):
    flist=[fnameOISSTAnom(yrlims, jj, smoothClim=True, meth=smoothmethod, win=windowhalfwid) for yrlims in ylimlistobs]
    fanom=xr.open_mfdataset(flist,decode_times=False,parallel=True)
    sst_an=_detrend(fanom.sst_an.values)
    fout=fnameOISSTAnomDetr([ylimlistobs[0][0], ylimobs[-1][-1]], jj, smoothClim=False, meth=None, win=1)
    sst_an.to_netcdf(fout,mode='w')